Hello! Welcome to the tutorial on running the PS50 Reaction Software. In this tutorial we will overview how to install, operate, and 

Hello! Welcome to the tutorial on running the PS50 Reaction Software. In this tutorial we will overview how to install, operate, and interpret code generated by this software. This software is  intended to track the progress of a color-changing reaction over time. It performs fundamental statistical interpretations on the reaction based on a iimage captured via a connected webcamera

First of all, we will need to download the "MasterRepo" Directory @  https://github.com/PS50-Spring2018/MasterRepo.git 
    

The In practice, two computers are needed, one for running the experiment (taking images using a built-in camera or an 
external webcam) and one for running the communication manager that detects the accumulation of image data and
initiates the data analysis.

In [ ]:
#you will need to install cv2, time, numpy, uuid, matplotlib, seaborne, colorsys, 
#glob, and csv

#run this to install these dependencies

import sys 
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install uuid
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install opencv-python

In [1]:
import sys 
import time 
import numpy as np
import seaborn
import glob
import uuid
import matplotlib
import csv
import cv2
import os

In this tutorial we will unpack the code from the PS50 package

First we will unpack the image capture script:  

In [2]:
def snap(n=0):
    '''
    Snaps an instant of the webcam and displays
    
    Parameters: 
        n: int       | Number used to ID camera to be used 

    Returns: 
        frame: array | Frame captured by webcam
    '''
    vc = cv2.VideoCapture(n)
    if vc.isOpened(): # try to get the first frame
        #key = cv2.waitKey(50)
        rval, frame = vc.read()
        cv2.imwrite("frame%d.jpg" % n, frame) #save the image as jpg 

    else:
        rval = False
        frame = None

    vc.release()
    return frame


def detect(self, initial_img):
    '''
    Thresholds image from webcam and fits contours

    Parameters: 
        initial_img: array | Inputted image from the webcam 
    
    Returns: 
        center[ind]: float | Center of the largest circle detected
        radii[ind]: float  | Radiius of the largest circle detected 
    ''' 
    
    gray = cv2.cvtColor(initial_img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY_INV)[1]
    _, cont,hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) #finds contours
    
    center=[]
    radii=[]
    for line in cont:
        temp=circle(line)
        center.append(temp[0])
        radii.append(temp[1])
     
    radii=np.array(radii)
    try:
        ind=np.argmax(radii) #gets index of largest circle detected
        return center[ind] , radii[ind]
    except ValueError:
        print(".............Warning: No Circle Detected.............")
    
 

def circle(cnt):  
    '''
    Fits contour to a circle

    Parameters: 
        cnt: array    | Inputted contour
    
    Returns: 
        center: float | Center of the largest circle detected
        radii: float  | Radiius of the largest circle detected 
    ''' 

    (x,y),radius = cv2.minEnclosingCircle(cnt) 
    center = (int(x),int(y))
    radius = int(radius)

    return center, radius 

Now we  load the script that manages the capture images at a higher level

In [3]:
class ImageCapture:
    """
    ImageCapture is the class the contains of all of the functions required to gather images and 
    to identify shapes in the images. 
    Parameters: 
        reaction_id:    string  | Reaction identifier. 
        interv:         float   | Time between pictures being taken.
        t:              float   | Total time of the experiment.
        dir_file:       string  | The path of the destination of the picture files.
        n:              int     | The camera number, usually 0 for built in camera and 1 for webcam.

    Notes:
    """
    def __init__(self, time, interv,rxn_id, dir_file, n):
        self.reaction_id=rxn_id
        self.interv = interv
        self.t = time
        self.dir_file = dir_file
        self.rxn_foldername = os.path.join(self.dir_file, str(self.reaction_id))
        self.n = n

    def run_image(self):
        '''
        Creates the reaction directory, if it does not already exists, and starts the process to take images
        for the set amount of time and iterations. 
        '''
        if not os.path.exists(self.rxn_foldername):
            os.makedirs(self.rxn_foldername) #makes a reaction directory

        for i in range(int(self.t/self.interv)): 
            tempM,tempV=self.iteration() #runs a single image process
            time.sleep(self.interv) #time intervals between trials
        print("ends the run_image")
        return

    def getTime(self):
        '''
        Gets the current time and saves it as an unique string
        Returns: 
            time: string | The time formatted YearMonthDayHourMinuteSecond, Ex: 20180721065911
        '''
        currentDT = datetime.datetime.now() #gets the current date and time
        time=currentDT.strftime('%Y%m%d%H%M%s') #formats the time 
        #time=str(uuid.uuid4())#for PCs/santi's computer
        return time



    def ObtainImage(self):
        '''
        Captures a single image, locally save the image, detects a beaker, and calculates color statistics
        Returns: 
            mean: array | Mean of the RGB colors in the image
            var: array  | Variance of the RGB colors in the image
        '''
        initial_img = snap(self.n) 
        name= self.getTime() #write raw image to a file
        direct=os.getcwd()

        cv2.imwrite("%sframe%s.jpg" % (direct,name), initial_img)  #reads back that image in the correct format
        img = cv2.imread("%sframe%s.jpg" % (direct,name)) 


        #reads in image to cv2 for shape detecting
        #img = img[:,:,::-1] # Change BGR to RGB format - Tim - Testing
        #print('***img',img[:2,:2,:])
        #detects the beaker location and size, adjusts size


        #cv2.imshow('image',img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

        center, radius = detect(self,img)
        radius = radius - int(0.1*radius)

        return name, img, center, radius

    def iteration(self):
        name, img, center, radius = self.ObtainImage()

        # Draw circle into image
        circle=cv2.circle(img,center,radius,(0,255,0),2)
        #circle = circle[:,:,::-1] #Change BGR to RGB format - Tim
        np.save(self.rxn_foldername+"/%s.npy" % (name),circle)
        img = img[:,:,::-1] #Change BGR to RGB format - Tim


        mask=np.zeros((int(img.shape[0]),int(img.shape[1]),3))


        #cv2.imshow("image",circle)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()


        #Sets the extreme bounds of the circle
        #this is used to detect boundaries and ensure that there is no out of bounds exceptions
        #this creates a mask to not include values outside of the circle 


        down=min(img.shape[0],center[1]+radius)
        up=max(center[1]-radius,0)


        print(up, down)
        for i in np.arange(up,down):
            #print(i)

            deltax=int(np.sqrt(np.abs(int(radius)**2-int(i-center[1])**2)))
            #deltax=int(np.sqrt(np.abs(int(radius)**2-int(i-center[1])**2)))

            left=center[0]-deltax
            right=center[0]+deltax
            #print("y: " + str(i) + " left: " + str(max(left,0)) + " right: "+ str(min(right,int(img.shape[0]))))
            left=max(left,0)
            right=min(right,int(img.shape[1]))
            x=np.arange(left,right)
            mask[i,x,:]=1


        #Applies mask


        #img_masked = np.multiply(img, mask)
        #cv2.imshow("mask",img_masked)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        


        img_nonzero = []

        # Goes through image and appends pixels that are in circle
        for row in range(mask.shape[0]):
            for col in range(mask.shape[1]):
                #Detects where in the mask nan values are present sorts it
                if int(mask[row,col,:].any()) == 1:
                    img_nonzero.append(img[row, col])

        img_nonzero = np.array(img_nonzero)


        mean=[np.mean(img_nonzero[:,0]),np.mean(img_nonzero[:,1]),np.mean(img_nonzero[:,2])]
        var=[np.std(img_nonzero[:,0]),np.std(img_nonzero[:,1]),np.std(img_nonzero[:,2])]

        # file to save the output of the program
        self.save(self.reaction_id,name,mean,var,self.rxn_foldername)

        return mean, var


    def save(self,rxnID, file, mean, variance, folderwoID):
        '''
        Saves csv file and writes statistics to it. 
        Parameters: 
            rxnID: float/string | The reaction identifier. 
            file: string        | File name of the image being added to the csv
            mean: array         | Mean of the RGB values in the image. 
            variance: array     | Variance of the RGB values. 
            folderwoID:  string | The file path to the directory where images are being saved. 
        '''
        with open(folderwoID+'/summary_%s.csv' % (rxnID),'a+') as csvfile:
            swriter = csv.writer(csvfile)
            swriter.writerow([file, mean[0],mean[1],mean[2], variance[0],variance[1],variance[2]])





Output from this  cells might look like this

In [4]:
from matplotlib import pyplot as plt
from matplotlib import image as img
from matplotlib import gridspec as grd
import seaborn as sns
import numpy as np
import colorsys as cs
import os

In [5]:

def dashboard(mean_RGB, var_RGB, image_array, N = 100):
    '''
    Display a dashboard containing the following information throughout the experiment:
    1: time trace of RGB values
    2: image of reaction flask
    3: position on color intensity bar
    4: history of actual color
    5: position on color wheel

    Parameters:
    mean_RGB: 		array 	| mean RGB values for all images taken up to the current point in the experiment
    var_RGB: 		array 	| variances in RGB values for all images taken up to the current point in the experiment
    image_array: 	array 	| latest image of the reaction flask
    N: 				int 	| number of points used to construct the color wheel and color intensity bar

    Notes:
    1: N > 200 is not recommended for most laptops.
    2: mean_RGB and var_RGB can be properly formatted by appending mean RBGs and RGB variances to an empty list, then using np.array
    '''

    # construct a 2-dimensional polar space wherein each point is a color (hue and saturation) in HSV space
    # then convert each point to RGB for plotting
    radii = np.linspace(0,1,N) 
    thetas = np.linspace(0,2*np.pi,N)
    t = [] 
    r = [] 
    c = []
    for theta in thetas:
        for radius in radii:
            t.append(theta)
            r.append(radius)
            c.append(cs.hsv_to_rgb(theta/(2*np.pi),radius,1)) # all HSV inputs must be 0-1

    # construct a 2-dimensional Cartesian space wherein each x-coordinate is a color intensity (value) in HSV space
    # then convert to RGB for plotting
    x_dim = np.linspace(0,1,N)
    y_dim = np.linspace(0,1,N)
    x = []
    y = []
    color = []
    for x_val in x_dim:
        for y_val in y_dim:
            x.append(x_val)
            y.append(y_val)
            color.append(cs.hsv_to_rgb(0, 0, 1-x_val)) # rescale for a light-to-dark gradient

    # allow the function to be called repeatedly to update the dashboard in real time
    plt.ion()
    plt.close('all')

    # construct a 3 X 6 grid for plotting
    gs =grd.GridSpec(3,6) 
    lines = plt.subplot2grid((3,6),(0,0), colspan=2, rowspan =2)
    beaker = plt.subplot2grid((3,6),(0,2), colspan=2, rowspan =2)
    colorbar = plt.subplot2grid((3,6),(2,4), colspan=2)
    squares = plt.subplot2grid((3,6),(2,0), colspan=4)
    colorwheel = plt.subplot2grid((3,6),(0,4), projection = 'polar', colspan=2, rowspan =2) # polar projection for HSV-based color construction

    # plot color wheel
    colorwheel.scatter(t, r, c=c, alpha=1.0) # alpha = 1 ensures accurate representation of colors
    colorwheel.xaxis.set_visible(False)
    colorwheel.yaxis.set_visible(False)
    colorwheel.set_title('Tracking through Color Space', fontsize = 8)
    colorwheel.axis('off')

    # plot color intensity bar
    colorbar.scatter(x, y, c=color, alpha=1.0) # alpha = 1 ensures accurate representation of color intensities
    colorbar.xaxis.set_visible(False)
    colorbar.yaxis.set_visible(False)
    colorbar.axis('off')
    colorbar.set_title('Tracking through Intensity Space', fontsize = 8)

    # plot mean RGB values over the history of the experiment with error bars representing variances
    line_colors = ['r','g','b'] 
    for i, c in enumerate(line_colors):
        lines.errorbar(range(len(mean_RGB)),mean_RGB[:,i],yerr=var_RGB[:,i],color=c)
    lines.set_title('History of Mean RGB Values', fontsize = 8)
    lines.set_xlabel('Iterations', fontsize = 8)

    # display latest image of the reaction flask
    beaker.imshow(image_array, interpolation='nearest') # interpolation = 'nearest' ensures image is displayed accurately
    beaker.axis('off')
    beaker.set_title('Latest Beaker Image', fontsize = 8)

    # plot path through color and intensity spaces
    t_val = []
    r_val = []
    v_val = []
    y_val = np.linspace(0,1,len(mean_RGB))
    for color in mean_RGB:
        r, g, b = color[0]/255, color[1]/255, color[2]/255 # all RGB values must be 0-1
        hsv = cs.rgb_to_hsv(r,g,b)
        t_val.append(hsv[0]*2*np.pi) # rescale 0-2*pi for polar plotting
        r_val.append(hsv[1])
        v_val.append(hsv[2])
    colorwheel.plot(t_val,r_val, 'k-')
    colorwheel.plot(t_val[-1],r_val[-1], 'ko')
    colorbar.plot(1-np.array(v_val),y_val,'y-')
    colorbar.plot(1-v_val[-1],y_val[-1],'yo') # plot latest points as circles to show the latest position

    # display succession of colors over the course of the experiment
    c_squares = mean_RGB/255 # rescale mean_RGB for the following plot
    for i in range(1,len(mean_RGB)+1):
        squares.plot([i-1+0.49,i-0.49], [0,0], '-', linewidth=100, c=c_squares[i-1])
    squares.axis('off')
    squares.set_title('Mean Color in the Beaker over Time',fontsize = 8)

    # display dashboard
    plt.tight_layout() # ensure that plots don't overlap on the dashboard
    plt.savefig('TEST.png')
    plt.show()
    plt.pause(0.05) # allows for the master script to run while the dashboard "waits" (pause) to be called again


In [6]:

class CommunicationManager(object):
    '''
    Class for handling communication with data acquisition, data analysis and output.

    Parameters:
            dir_file: string      | Directory containing image data
            rxd_id: string or int | Reaction ID
    '''

    def __init__(self, dir_file, rxn_id):
        '''
        Initializes the CommunicationManager class.
        '''
        self.dir_file = dir_file
        self.reaction_id = rxn_id

        self.csvname = 'summary_{}.csv'.format(rxn_id) #name of experiment summary file
        self.processed_indices = [0] # list of image indices that have been processed
        self.means = [] # list of RGB means
        self.variances = [] # list of RGB variances


    def initialize(self):
        '''
        Change to image directory.
        '''
        path_data = os.path.join(self.dir_file, self.reaction_id)
        os.chdir(path_data)



    def run_comms(self):
        '''
        Runs analysis loop: continuously checks for new images, loads them and plots dashboard.
        Exit analysis loop with ctrl-c.
        '''

        # Structure to exit analysis loop with ctrl-c
        try:
            # Continuously check for new images
            while True:

                # Try loading data and plotting dashboard
                try:
                    # Load data from current image
                    mean_array, var_array, image_array = self.load_data()

                    # Append to mean and variance list for all images
                    self.means.append(mean_array)
                    self.variances.append(var_array)

                    # Plot dashboard
                    dashboard(np.array(self.means), np.array(self.variances), image_array) 

                    # Update which images have been processes
                    self.processed_indices.append(self.processed_indices[-1]+1) 

                    # Pause to wait for new images to be collected
                    time.sleep(2.)

                # If we ran out of images: pause to wait for new images
                except IndexError:
                    print('Waiting for new images...')
                    time.sleep(5)

        except KeyboardInterrupt:
            print('\nCommunicationManager closed by user')
            pass


    def load_data(self):
        '''
        Loads data from image and summary files.

        Returns:
            mean_array: array  | RGB value means of current image
            var_array: array   | RGB value variances of current image
            image_array: array | Current image in array format
        '''

        # Get current time stamps
        timestamps = self.gettimestamp()   

        # Load the next image         
        last_img_index = self.processed_indices[-1]
        image_array = np.load('{}.npy'.format(timestamps[last_img_index]))

        # Load data from summary csv file
        #csvfile = open(self.csvname, 'r')
        with open(self.csvname, 'r') as csvfile:
            reader = csv.reader(csvfile)
            my_csv_data = list(reader)      
        #csvfile.close()
        data = my_csv_data[last_img_index] # grabs the mean & variance data of the current image

        # Create arrays of RGB value means and variances
        mean_array = [float(data[1]), float(data[2]), float(data[3])] 
        var_array = [float(data[4]), float(data[5]), float(data[6])]

        return mean_array, var_array, image_array


    def gettimestamp(self):
        ''' 
        Creates list of timestamps of files in the directory. 
        '''

        timestamps = []

        for file in glob.glob("*.npy"): # "for every .npy file in the current directory "
            name = file.split('.') 
            # grab the timestamp portion 
            timestamp_str = name[0] 
            # convert the timestamp string into an integer
            timestamp_int = int(timestamp_str) 
            # append to list of timestamps
            timestamps.append(timestamp_int) 

        # sorts the timestamps in increasing order
        timestamps.sort() 

        return timestamps



The main function here runs the show

In [7]:
""" Script to run experiment on the image acquisition computer """

#from ImageCapture import ImageCapture


if __name__=='__main__':
    """Runs experiment.

    Parameters:
        dir_file      : string         |   Directory path to image files
        reaction_id   : float/string   |   Reaction identifer
        duration      : int            |   The total time the user would like to take images and analyze for
        img_interval  : int            |   The interval between image captures
        camera_number : int            |   The camera number, 0 for built-in camera and 1 for external webcam
    """ 

    # File path to directory for placing image data
    dir_file = input("Enter filepath to experiment directory : ")

    # Reaction ID (Note: this will become the name of the experiment directory)
    reaction_id = input("Enter Reaction ID : ")

    # Duration of experiment, in seconds (time window to take images)
    duration = int(input("Enter duration of experiment [s] : "))
    
    # Time interval between images
    img_interval = int(input("Enter interval between images : "))

    # Type of camera (built-in camera or external webcam)
    camera_number = int(input("Enter camera number (0 = built-in camera, 1 = external webcam) : "))

    # Creates instance of ImageCapture event and runs experiment
    img_expt = ImageCapture(duration, img_interval, reaction_id, dir_file, camera_number)
    img_expt.run_image()

Enter filepath to experiment directory : 
Enter Reaction ID : jupyter_testing
Enter duration of experiment [s] : 10
Enter interval between images : 10
Enter camera number (0 = built-in camera, 1 = external webcam) : 0
0 480
ends the run_image


In [ ]:
#now the dashboard display

In [8]:
from CommunicationManager import CommunicationManager


if __name__=='__main__':
    """Runs data analysis and plotting.

    Parameters:
        ...
    """ 

    # File path to image data
    dir_file = input("Enter filepath to experiment directory : ")
    #dir_file = raw_input("Enter filepath to experiment directory : ")

    # Reaction ID (Note: this is the name of the experiment directory)
    reaction_id = input("Enter Reaction ID : ")
    #reaction_id = raw_input("Enter Reaction ID : ")


    # Creates instance of CommunicationManager class and runs analysis
    manager = CommunicationManager(dir_file, reaction_id)
    manager.initialize()
    manager.run()


Enter filepath to experiment directory : 
Enter Reaction ID : jupyter_testing


ValueError: invalid literal for int() with base 10: '2cb8e6bc-10d5-4c7c-94c4-0fdc515d1c6e'